# Clustering using Genetic Algorithm

## Introduction
This Python script implements a genetic algorithm to perform clustering on the Iris dataset. The genetic algorithm optimizes the assignment of data points to clusters by maximizing a fitness metric based on minimizing the Euclidean distance within clusters. The algorithm utilizes selection, crossover, and mutation operations over multiple generations to evolve a population of cluster assignments.

### Import dependencies

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import random, randint, sample
from sklearn.metrics import accuracy_score
from scipy.optimize import linear_sum_assignment
from sklearn.preprocessing import LabelEncoder

### Population Generation
This function initializes a population of individuals where each individual represents a potential solution (cluster assignment). It randomly assigns data points to clusters based on the specified number of clusters (num_clusters) and the size of the dataset (genome_size).

In [21]:
def initialize_population(population_size):
    population = []
    for _ in range(population_size):
        individual = [randint(1, num_clusters) for _ in range(genome_size)]
        population.append(individual)
    return population

### Calculate Euclidean distance

In [22]:
def euclidean_distance(point1, point2):
    return np.sqrt(np.sum((np.array(point1) - np.array(point2)) ** 2))

### Find Cluster Center

In [23]:
def find_cluster_center(cluster):
    return np.mean(cluster, axis=0)

### Fitness
The fitness function calculates the fitness of an individual (cluster assignment) by computing the inverse of the sum of Euclidean distances within clusters. It first organizes data points into clusters based on the provided individual, computes the centroid for each cluster, and then measures the average distance of data points to their respective centroids.

In [24]:
def fitness(individual, df):
    clusters = {i: [] for i in range(1, num_clusters + 1)}
    for i, cluster_id in enumerate(individual):
        clusters[cluster_id].append(df.iloc[i].values)
    
    sum_of_distances = 0
    for cluster_points in clusters.values():
        if cluster_points:
            cluster_center = find_cluster_center(cluster_points)
            for point in cluster_points:
                sum_of_distances += euclidean_distance(cluster_center, point)
    
    return 1 / sum_of_distances if sum_of_distances > 0 else float('inf')


### Selection, Mutation, Cross-over
The selection function chooses parent individuals from the population based on their fitness scores, using tournament selection to ensure that individuals with higher fitness have a higher chance of being selected. Crossover combines genetic material from two parent individuals to create offspring, promoting genetic diversity. Mutation introduces random changes to individual solutions, helping explore new possibilities in the search space.

In [25]:
def selection(population, fitnesses):
    tournament = [population[randint(0, len(population) - 1)] for _ in range(tournament_size)]
    tournament_fitnesses = [fitnesses[population.index(ind)] for ind in tournament]
    return tournament[np.argmax(tournament_fitnesses)]

def crossover(parent1, parent2):
    crossover_point1, crossover_point2 = sorted(sample(range(genome_size), 2))
    child1 = parent1[:crossover_point1] + parent2[crossover_point1:crossover_point2] + parent1[crossover_point2:]
    child2 = parent2[:crossover_point1] + parent1[crossover_point1:crossover_point2] + parent2[crossover_point2:]
    return child1, child2

def mutate(individual):
    if random() < mutation_rate:
        mutation_point = randint(0, genome_size - 1)
        individual[mutation_point] = randint(1, num_clusters)

### Calculate clustering accuracy

In [26]:
def calculate_accuracy(individual, true_labels):
    individual = np.array(individual) - 1
    encoder = LabelEncoder()
    true_labels = encoder.fit_transform(true_labels)
    
    cost_matrix = np.zeros((num_clusters, num_clusters))
    for i in range(num_clusters):
        for j in range(num_clusters):
            cost_matrix[i, j] = np.sum((individual == i) & (true_labels == j))
    
    row_ind, col_ind = linear_sum_assignment(-cost_matrix)
    accuracy = np.sum(individual == col_ind[true_labels]) / len(true_labels)
    
    return round(accuracy*100, 2)

### Generic Algorithms
The main genetic_algorithm function orchestrates the entire clustering process using a genetic algorithm approach. It initializes a population of cluster assignments, iterates through a specified number of generations, evaluates fitness, performs selection, crossover, and mutation operations to evolve the population, and tracks the best fitness and accuracy achieved. At the end of execution, it displays plots showing the evolution of fitness and accuracy over generations and returns the best clustering solution found.

In [27]:
def genetic_algorithm(population_size):
    population = initialize_population(population_size)
    best_individual = None
    best_fitness = -float('inf')
    fitness_over_time = []
    accuracy_over_time = []

    for generation in range(max_generations):
        fitnesses = [fitness(ind, df) for ind in population]
        
        new_population = []
        for _ in range(len(population) // 2):
            parent1 = selection(population, fitnesses)
            parent2 = selection(population, fitnesses)
            child1, child2 = crossover(parent1, parent2)
            mutate(child1)
            mutate(child2)
            new_population.extend([child1, child2])
        
        population = new_population

        current_best_fitness = max(fitnesses)
        fitness_over_time.append(current_best_fitness)
        
        current_best_individual = population[np.argmax(fitnesses)]
        current_accuracy = calculate_accuracy(current_best_individual, labels)
        accuracy_over_time.append(current_accuracy)
        
        if current_best_fitness > best_fitness:
            best_fitness = current_best_fitness
            best_individual = current_best_individual
            best_accuracy = current_accuracy
        

        print(f"Generation {generation+1}: Best Fitness = {best_fitness}, Accuracy = {current_accuracy}")

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(fitness_over_time)
    plt.xlabel('Generation')
    plt.ylabel('Fitness')
    plt.title('Fitness Over Generations')
    
    plt.subplot(1, 2, 2)
    plt.plot(accuracy_over_time)
    plt.xlabel('Generation')
    plt.ylabel('Accuracy')
    plt.title('Accuracy Over Generations')
    
    plt.show()

    print(f"best fitness = {best_fitness} for population size of {population_size}.")

    return best_individual, best_fitness,best_accuracy, accuracy_over_time, fitness_over_time


### Run Genetic Algorithm with Tuning Population Size
The run_genetic_algorithm_with_tuning function iterates over a list of population sizes and executes the genetic algorithm for each size, collecting the best individual, fitness, accuracy, and the evolution of these metrics over generations. It plots the fitness and accuracy trends for each population size, providing insights into how different sizes affect algorithm performance.

In [28]:
def run_genetic_algorithm_with_tuning(population_sizes):
    results = []
    for pop_size in population_sizes:
        print(f"Running genetic algorithm with population size = {pop_size}")
        best_individual, best_fitness, best_accuracy, accuracy_over_time, fitness_over_time = genetic_algorithm(pop_size)
        results.append((pop_size, best_individual, best_fitness, best_accuracy, accuracy_over_time, fitness_over_time))
    
    # Plotting results
    plt.figure(figsize=(15, 10))

    plt.subplot(2, 1, 1)
    for result in results:
        plt.plot(result[4], label=f"Population Size = {result[0]}")
    plt.xlabel('Generation')
    plt.ylabel('Best Fitnesses')
    plt.title('Fitness Over Generations')
    plt.legend()

    plt.subplot(2, 1, 2)
    for result in results:
        plt.plot(result[5], label=f"Population Size = {result[0]}")
    plt.xlabel('Generation')
    plt.ylabel('Best Accuracies')
    plt.title('Accuracy Over Generations')
    plt.legend()

    plt.tight_layout()
    plt.show()

    # Return all results
    return results


## Main

In [34]:
# Load and prepare the data
df = pd.read_csv("./iris.data", header=None)
df.columns = ["sepal length", "sepal width", "petal length", "petal width", "class"]
labels = df['class'].values
df.drop("class", axis=1, inplace=True)

# Normalize the dataset
df = (df - df.mean()) / df.std()

# Parameters
genome_size = len(df)
num_clusters = 3
max_generations = 200
mutation_rate = 0.008
tournament_size = 5

# Define the population sizes to tune
population_sizes_to_tune = [10, 20, 30 , 40, 50]

# Run the genetic algorithm with different population sizes and visualize the results
results = run_genetic_algorithm_with_tuning(population_sizes_to_tune)

# Find and print the best result
best_result = max(results, key=lambda x: x[3])  
best_individual = best_result[1]
best_fitness = best_result[2]
best_accuracy = best_result[3]

print("\nBest Result:")
print("Population Size:", best_result[0])
print("Best Individual (Cluster Assignment):\n", best_individual)
print("Best Fitness:", best_fitness)
print("Best Accuracy:", best_accuracy)

Running genetic algorithm with population size = 10
Generation 1: Best Fitness = 0.0036877327611898247, Accuracy = 33.33
Generation 2: Best Fitness = 0.0036877327611898247, Accuracy = 42.67
Generation 3: Best Fitness = 0.0036877327611898247, Accuracy = 42.67
Generation 4: Best Fitness = 0.0036877327611898247, Accuracy = 42.67
Generation 5: Best Fitness = 0.0036877327611898247, Accuracy = 42.67
Generation 6: Best Fitness = 0.0036877327611898247, Accuracy = 42.67
Generation 7: Best Fitness = 0.0036877327611898247, Accuracy = 42.67
Generation 8: Best Fitness = 0.0036877327611898247, Accuracy = 42.67
Generation 9: Best Fitness = 0.0036877327611898247, Accuracy = 42.67
Generation 10: Best Fitness = 0.0036877327611898247, Accuracy = 42.67
Generation 11: Best Fitness = 0.0036877327611898247, Accuracy = 42.67
Generation 12: Best Fitness = 0.0036877327611898247, Accuracy = 42.67
Generation 13: Best Fitness = 0.0036877327611898247, Accuracy = 42.67
Generation 14: Best Fitness = 0.003687732761189

Traceback (most recent call last):
  File "C:\Users\Aidin\Desktop\uni4022\AI\DecisionTree\env\Lib\site-packages\IPython\core\interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Aidin\AppData\Local\Temp\ipykernel_12688\3881618803.py", line 21, in <module>
    results = run_genetic_algorithm_with_tuning(population_sizes_to_tune)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Aidin\AppData\Local\Temp\ipykernel_12688\2744423964.py", line 5, in run_genetic_algorithm_with_tuning
    best_individual, best_fitness, best_accuracy, accuracy_over_time, fitness_over_time = genetic_algorithm(pop_size)
                                                                                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Aidin\AppData\Local\Temp\ipykernel_12688\2335260082.py", line 9, in genetic_algorithm
    fitnesses = [fitness(ind, df) for ind in population]
                 ^